# EDAした結果よりデータを変形するアレ
開発時以外は**`Run ALL`以外しない**

## テスト通らないカラムを扱っちゃだめよ

In [5]:
# `python -m unittest discover tests -v`と等価
import unittest
TEST_RESULT = unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().discover("tests")
)

test_error (test_embarked_at_cherbourg.TestEmbarkedAtCherbourg)
エラーを出す場合のテスト ... ok
test_main (test_embarked_at_cherbourg.TestEmbarkedAtCherbourg)
メインのテスト ... ok
test_error (test_int_sex.testIntSex)
エラーになるケースのテスト ... ok
test_main (test_int_sex.testIntSex)
メインのテスト ... ok
test_error (test_is_pair.testIsPair)
エラーを出す場合のテスト ... ok
test_example (test_is_pair.testIsPair)
例なデータでのテスト ... ok
test_Age_not_found (test_under_10_years_old.TestUnder10YearsOld)
Ageカラムが存在しない場合のエラーのテスト ... ok
test_all_age (test_under_10_years_old.TestUnder10YearsOld)
0-200才までテスト ... ok
test_example (test_under_10_years_old.TestUnder10YearsOld)
例なデータのテスト（正例） ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.050s

OK


## データの生成

In [6]:
import pandas as pd

from transforms.column_under_10_years_old import column_under_10_years_old
from transforms.column_is_pair import column_is_pair
from transforms.column_int_sex import column_int_sex
from transforms.column_embarked_at_cherbourg import column_embarked_at_cherbourg


def generate(raw: pd.DataFrame) -> pd.DataFrame:
    preprocessed = pd.DataFrame(
        {
            "PassengerId": raw.PassengerId,
            "IntSex": column_int_sex(raw),
            "Pclass": raw.Pclass,
            "Under10YearsOld": column_under_10_years_old(raw),
            "IsPair": column_is_pair(raw),
            "Parch": raw.Parch,
            "EmbarkedAtCherboug": column_embarked_at_cherbourg(raw),
        }
    )

    # ターゲットの列があれば含める（TRAIN向け）
    if "Survived" in raw:
        return preprocessed.assign(Survived=raw.Survived)

    return preprocessed


In [7]:
if (
    len(TEST_RESULT.failures) == 0
    and len(TEST_RESULT.errors) == 0
    and TEST_RESULT.testsRun != 0
):
    TRAIN, TEST = (
        generate(pd.read_csv("data/train.csv")),
        generate(pd.read_csv("data/test.csv")),
    )
    train_filename = "data/preprocessed_train.csv"
    test_filename = "data/preprocessed_test.csv"

    print("following dataframe will be saved (or OVERWRITED)")
    print("-----------", train_filename, "-----------")
    display(TRAIN)

    print("-----------", test_filename, "-----------")
    display(TEST)

    if input("continue to save (overwrite)? (y/N)"):
        TRAIN.to_csv(train_filename, index=None, header=True)
        TEST.to_csv(test_filename, index=None, header=True)
else:
    print("test is not completed successfully or nothing tested.")


following dataframe will be saved (or OVERWRITED)
----------- data/preprocessed_train.csv -----------


,PassengerId,IntSex,Pclass,Under10YearsOld,IsPair,Parch,EmbarkedAtCherboug,Survived
0,1,0,3,0,1,0,0,0
1,2,1,1,0,1,0,1,1
2,3,1,3,0,0,0,0,1
3,4,1,1,0,1,0,0,1
4,5,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
886,887,0,2,0,0,0,0,0
887,888,1,1,0,0,0,0,1
888,889,1,3,0,1,2,0,0
889,890,0,1,0,0,0,1,1


----------- data/preprocessed_test.csv -----------


,PassengerId,IntSex,Pclass,Under10YearsOld,IsPair,Parch,EmbarkedAtCherboug
0,892,0,3,0,0,0,0
1,893,1,3,0,1,0,0
2,894,0,2,0,0,0,0
3,895,0,3,0,0,0,0
4,896,1,3,0,1,1,0
...,...,...,...,...,...,...,...
413,1305,0,3,0,0,0,0
414,1306,1,1,0,0,0,1
415,1307,0,3,0,0,0,0
416,1308,0,3,0,0,0,0
